In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from math import log, sqrt

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 
              'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict, parse_dates=['date'])

In [3]:
def get_X_y(df, features, output):
    X = np.c_[np.ones(df.shape[0]), df[features].values]
    y = df[output].values
    return X, y

def predict_output(feature_matrix, weights):
    pred = np.matmul(feature_matrix, weights.T)    
    return pred
    
def normalize_features(X):
    norms = np.linalg.norm(X, axis=0)
    X_normalized = X / norms
    return X_normalized, norms

def rho(j, X, y, weights):
    rho = np.dot(X[:, j], (y - predict_output(X, weights) + X[:, j]*weights[j]))
    return rho

def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute rho[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    rho_i = rho(i, feature_matrix, output, weights)
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = rho_i
    elif rho_i < -l1_penalty/2.:
        new_weight_i = rho_i + l1_penalty/2.
    elif rho_i > l1_penalty/2.:
        new_weight_i = rho_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, 
                                      l1_penalty, tolerance):
    weights = initial_weights
    diff_weights = [float('inf') for _ in range(len(weights))]
    while max(diff_weights) > tolerance:
        new_weights = np.copy(weights)
        for i in range(len(weights)):
            new_weight_i = lasso_coordinate_descent_step(i, feature_matrix, 
                                                         output, new_weights, l1_penalty)
            new_weights[i] = new_weight_i
        diff_weights = [abs(x - y) for x, y in zip(new_weights, weights)]
        weights = np.array(new_weights)
#         print(weights)
    return weights

In [4]:
X, y = get_X_y(sales, ['sqft_living', 'bathrooms'], 'price')
X_normalized, norms = normalize_features(X)
weights = np.array([1., 4., 1.])
print('{:e}'.format(2*rho(1, X_normalized, y, weights)))
print('{:e}'.format(2*rho(2, X_normalized, y, weights)))
learned_weights = lasso_cyclical_coordinate_descent(X_normalized, y, initial_weights=weights, 
                                      l1_penalty=1e7, tolerance=1.)
print(learned_weights)

1.758789e+08
1.686109e+08
[ 21624997.96137046  63157247.20619597         0.        ]


In [5]:
rss = np.square(predict_output(feature_matrix=X_normalized, weights=learned_weights) - y).sum()
print(rss)

1.63049247673e+15


In [6]:
train = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict, parse_dates=['date'])
train = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict, parse_dates=['date'])

In [7]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
            'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 
            'yr_renovated']
X, y = get_X_y(train, features, 'price')
weights = np.zeros(len(features)+1)
X_normalized, norms = normalize_features(X)
weights1e7 = lasso_cyclical_coordinate_descent(X_normalized, y, initial_weights=weights, 
                                      l1_penalty=1e7, tolerance=1.)
for i in range(len(features)):
    print(features[i], weights1e7[i+1])

bedrooms 0.0
bathrooms 0.0
sqft_living 6211298.91051
sqft_lot 0.0
floors 0.0
waterfront 0.0
view 4034298.43186
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


In [8]:
weights1e8 = lasso_cyclical_coordinate_descent(X_normalized, y, initial_weights=weights, 
                                      l1_penalty=1e8, tolerance=1.)
for i in range(len(features)):
    print(features[i], weights1e8[i+1])

bedrooms 0.0
bathrooms 0.0
sqft_living 0.0
sqft_lot 0.0
floors 0.0
waterfront 0.0
view 0.0
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


In [9]:
weights1e4 = lasso_cyclical_coordinate_descent(X_normalized, y, initial_weights=weights, 
                                      l1_penalty=1e4, tolerance=5e5)
for i in range(len(features)):
    print(features[i], weights1e4[i+1])

bedrooms -1003478.40659
bathrooms 13603866.1149
sqft_living 18236373.7379
sqft_lot -1486948.41802
floors -6909729.83159
waterfront 2610963.45337
view 4601807.14304
condition -9674853.97344
grade 2721542.80438
sqft_above 10824915.8951
sqft_basement 1307362.20656
yr_built -27532090.7729
yr_renovated 1479911.28195


In [10]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms

In [11]:
rss1e7 = np.square(predict_output(feature_matrix=X, 
                                  weights=normalized_weights1e7) - y).sum()
print(rss1e7)

4.07721622772e+14


In [12]:
rss1e8 = np.square(predict_output(feature_matrix=X, 
                                  weights=normalized_weights1e8) - y).sum()
print(rss1e8)

5.37108649657e+14


In [13]:
rss1e4 = np.square(predict_output(feature_matrix=X, 
                                  weights=normalized_weights1e4) - y).sum()
print(rss1e4)

2.503445952e+14
